In [1]:
from yfinance import download

In [2]:
symbols = {'AMZN': 'USD', 
           'BT-A.L': 'GBP', 
           'ULVR.L': 'GBP', 
           'MTS.MC': 'EUR', 
           'MSFT': 'USD'}

tickers = list(symbols.keys())

In [3]:
data = download(tickers, auto_adjust=True)

[*********************100%***********************]  5 of 5 completed


In [4]:
quotes = data['Close']['2020':]
quotes

,AMZN,BT-A.L,MSFT,MTS.MC,ULVR.L
Date,,,,,
2020-01-02,1898.010010,196.059998,158.571075,15.842000,4167.220703
2020-01-03,1874.969971,193.699997,156.596588,15.282000,4176.807129
2020-01-06,1902.880005,193.240005,157.001373,15.004000,4145.651855
2020-01-07,1906.859985,192.080002,155.569855,15.054000,4110.181641
2020-01-08,1891.969971,195.899994,158.047836,15.240000,4064.645752
...,...,...,...,...,...
2021-04-05,3226.729980,NaN,249.070007,NaN,NaN
2021-04-06,3223.820068,152.649994,247.860001,24.780001,4088.500000
2021-04-07,3279.389893,156.399994,249.899994,24.434999,4133.500000


In [5]:
currency = ['EURUSD=X', 'EURGBP=X']
data_currency = download(currency, auto_adjust=True)

[*********************100%***********************]  2 of 2 completed


In [6]:
quotes_currency = data_currency['Close'].reindex(quotes.index).ffill()
quotes_currency.columns = quotes_currency.columns.str[3:6]
quotes_currency['EUR'] = 1
quotes_currency

,GBP,USD,EUR
Date,,,
2020-01-02,0.84657,1.122083,1
2020-01-03,0.84950,1.117144,1
2020-01-06,0.85329,1.116196,1
2020-01-07,0.84998,1.119799,1
2020-01-08,0.85053,1.115474,1
...,...,...,...
2021-04-05,0.85070,1.176817,1
2021-04-06,0.84931,1.181893,1
2021-04-07,0.85870,1.187423,1


In [7]:
quotes_currency_data = quotes_currency.reindex([symbols[f] for f in quotes.columns], axis=1)
quotes_currency_data.columns = quotes.columns
quotes_currency_data

,AMZN,BT-A.L,MSFT,MTS.MC,ULVR.L
Date,,,,,
2020-01-02,1.122083,0.84657,1.122083,1,0.84657
2020-01-03,1.117144,0.84950,1.117144,1,0.84950
2020-01-06,1.116196,0.85329,1.116196,1,0.85329
2020-01-07,1.119799,0.84998,1.119799,1,0.84998
2020-01-08,1.115474,0.85053,1.115474,1,0.85053
...,...,...,...,...,...
2021-04-05,1.176817,0.85070,1.176817,1,0.85070
2021-04-06,1.181893,0.84931,1.181893,1,0.84931
2021-04-07,1.187423,0.85870,1.187423,1,0.85870


In [8]:
quotes_eur = quotes.div(quotes_currency_data)
quotes_eur

,AMZN,BT-A.L,MSFT,MTS.MC,ULVR.L
Date,,,,,
2020-01-02,1691.506512,231.593364,141.318542,15.842000,4922.476144
2020-01-03,1678.360525,228.016477,140.175862,15.282000,4916.782962
2020-01-06,1704.790147,226.464626,140.657526,15.004000,4858.432363
2020-01-07,1702.858408,225.981791,138.926527,15.054000,4835.621611
2020-01-08,1696.113219,230.326958,141.686723,15.240000,4778.956200
...,...,...,...,...,...
2021-04-05,2741.913822,NaN,211.647240,NaN,NaN
2021-04-06,2727.673987,179.734134,209.714333,24.780001,4813.907869
2021-04-07,2761.770866,182.135784,210.455769,24.434999,4813.671956


### Hagamos que sea una función, y que averigue las divisas pasando solo los tickers

In [9]:
from yfinance import download
from requests import get
from bs4 import BeautifulSoup
from re import compile as re_compile

def get_currency(symbol: str):
    url = f'https://finance.yahoo.com/quote/{symbol}'
    response = get(url)
    soup = BeautifulSoup(response.content)
    return soup.find(text=re_compile(r"\bCurrency in \b")).split()[-1]

def symbols_currecy(symbols: list):
    currencies = {}
    for symbol in symbols:
        currencies[symbol] = get_currency(symbol).upper()
    return currencies

def download_to_currency(tickers: list, 
                         to_currency: str = 'EUR', 
                         from_date: str = '2010-01-01'):

    symbols = symbols_currecy(tickers)
    print(symbols)
    data = download(tickers, auto_adjust=True)
    quotes = data['Close'][from_date:]
    currency_symbols = [f'{to_currency}{f}=X' for f in list(set(symbols.values())) if f != to_currency]
    print(f'Download {currency_symbols}')
    data_currency = download(currency_symbols, auto_adjust=True)
    quotes_currency = data_currency['Close'].reindex(quotes.index).ffill()
    quotes_currency.columns = quotes_currency.columns.str[3:6]
    quotes_currency[to_currency] = 1
    quotes_currency_data = quotes_currency.reindex([symbols[f] for f in quotes.columns], axis=1)
    quotes_currency_data.columns = quotes.columns
    return quotes, quotes.div(quotes_currency_data)

In [10]:
tickers = ['AMZN', 'BT-A.L', 'ULVR.L', 'MTS.MC', 'MSFT']
to_currency = 'EUR'

quotes, quotes_eur = download_to_currency(tickers, 
                                          to_currency=to_currency, 
                                          from_date='2015')

{'AMZN': 'USD', 'BT-A.L': 'GBP', 'ULVR.L': 'GBP', 'MTS.MC': 'EUR', 'MSFT': 'USD'}
[*********************100%***********************]  5 of 5 completed
Download ['EURGBP=X', 'EURUSD=X']
[*********************100%***********************]  2 of 2 completed


In [11]:
quotes

,AMZN,BT-A.L,MSFT,MTS.MC,ULVR.L
Date,,,,,
2015-01-02,308.519989,307.779663,41.443684,17.861666,2136.597412
2015-01-05,302.190002,301.178833,41.062580,17.189480,2109.541260
2015-01-06,295.290009,296.189941,40.459892,17.185585,2110.360840
2015-01-07,298.420013,301.946381,40.973934,17.167665,2124.299805
2015-01-08,300.459991,306.474823,42.179317,17.815989,2187.429688
...,...,...,...,...,...
2021-04-05,3226.729980,NaN,249.070007,NaN,NaN
2021-04-06,3223.820068,152.649994,247.860001,24.780001,4088.500000
2021-04-07,3279.389893,156.399994,249.899994,24.434999,4133.500000


In [12]:
quotes_eur

,AMZN,BT-A.L,MSFT,MTS.MC,ULVR.L
Date,,,,,
2015-01-02,255.198477,396.674392,34.280971,17.861666,2753.702017
2015-01-05,252.954177,385.395439,34.372253,17.189480,2699.418054
2015-01-06,247.331961,378.565870,33.888802,17.185585,2697.291421
2015-01-07,251.293521,384.919673,34.503330,17.167665,2708.046321
2015-01-08,253.852631,391.261113,35.636461,17.815989,2792.582326
...,...,...,...,...,...
2021-04-05,2741.913822,NaN,211.647240,NaN,NaN
2021-04-06,2727.673987,179.734134,209.714333,24.780001,4813.907869
2021-04-07,2761.770866,182.135784,210.455769,24.434999,4813.671956


In [13]:
tickers = ['AMZN', 'BT-A.L', 'ULVR.L', 'MTS.MC', 'MSFT']
to_currency = 'USD'

quotes, quotes_usd = download_to_currency(tickers, 
                                          to_currency=to_currency, 
                                          from_date='2015')

{'AMZN': 'USD', 'BT-A.L': 'GBP', 'ULVR.L': 'GBP', 'MTS.MC': 'EUR', 'MSFT': 'USD'}
[*********************100%***********************]  5 of 5 completed
Download ['USDEUR=X', 'USDGBP=X']
[*********************100%***********************]  2 of 2 completed


In [14]:
quotes_usd

,AMZN,BT-A.L,MSFT,MTS.MC,ULVR.L
Date,,,,,
2015-01-02,308.519989,479.512135,41.443684,21.593706,3328.759207
2015-01-05,302.190002,460.349149,41.062580,20.535296,3224.414925
2015-01-06,295.290009,451.936185,40.459892,20.517897,3220.056770
2015-01-07,298.420013,457.085914,40.973934,20.387214,3215.761406
2015-01-08,300.459991,463.008850,42.179317,21.087005,3304.673754
...,...,...,...,...,...
2021-04-05,3226.729980,NaN,249.070007,NaN,NaN
2021-04-06,3223.820068,212.397378,247.860001,29.287320,5688.743630
2021-04-07,3279.389893,216.320880,249.899994,29.014676,5717.150840
